In [ ]:
from lip_conv.bounds import *
import torch
import numpy as np

# Spectral norm estimation on dense matrix

This code is related to dense spectral norm estimation, see Section 5.5 in paper. 

In [2]:
n, m = 500, 400
with_cuda = False
G = torch.randn(n, m).double().cuda()
sigma_1_reference = torch.linalg.matrix_norm(G, ord=2).item()
print("dense sigma_1_reference", sigma_1_reference)

dense sigma_1_reference 42.04387409662597


# Test Power iteration on dense matrix

In [3]:
n_iter_pi = 1000
nb_reps = 10
burn = 10
sigmas_1_pi, times = [], []
for _ in range(nb_reps + burn):
    sigma_1_pi, time = estimate_dense(G, 
                                      n_iter=n_iter_pi, 
                                      name_func="pi", 
                                      return_time=True)
    sigmas_1_pi.append(sigma_1_pi.item())
    times.append(time)
print("Diff Power iteration", np.mean(sigmas_1_pi[burn:]) - sigma_1_reference, 
      "Mean time", np.mean(times[burn:]))

Diff Power iteration -2.291876910476276e-09 Mean time 0.09564363956451416


# Test Gram iteration on dense matrix

In [4]:
n_iter_gram = 15
nb_reps = 10
burn = 10
sigmas_1_gram, times = [], []
for _ in range(nb_reps + burn):
    sigma_1_gram, time = estimate_dense(G, 
                                        n_iter=n_iter_gram, 
                                        name_func="ours", 
                                        return_time=True)
    sigmas_1_gram.append(sigma_1_gram.item())
    times.append(time)
print("Diff Gram iteration", np.mean(sigmas_1_gram[burn:]) - sigma_1_reference, 
      "Mean time", np.mean(times[burn:]))

Diff Gram iteration -3.225864020350855e-12 Mean time 0.0024421215057373047


# Spectral norm estimation on convolutional layer

This code is related to convolutional layer spectral norm estimation, see Section 5.2 in paper.# Define convolution kernel

In [5]:
cout = 64
cin = 64
input_size_n = 12
kernel_size = 5


kernel = torch.randn(cout, cin, kernel_size, kernel_size).cuda()

sigma_1_reference_sedghi2019 = estimate(kernel, 
                                        n=input_size_n, 
                                        name_func="sedghi2019").item()
print("kernel sigma_1_reference_sedghi2019", sigma_1_reference_sedghi2019)

n_iter_ryu2019_ref = 100
sigma_1_reference_ryu2019 = estimate(kernel, 
                                     n=input_size_n, 
                                     n_iter=n_iter_ryu2019_ref, 
                                     name_func="ryu2019").item()
print("kernel sigma_1_reference_ryu2019", sigma_1_reference_ryu2019)

kernel sigma_1_reference_sedghi2019 80.78829193115234
kernel sigma_1_reference_ryu2019 77.66583251953125


# Test Araujo2021 on convolutional layer

In [6]:
nb_samples_araujo2021 = 50
nb_reps = 10
burn = 10
sigmas_1_araujo2021, times = [], []
for _ in range(nb_reps + burn):
    sigma_1_araujo2021, time = estimate(kernel, 
                                  n=input_size_n, 
                                  n_iter=nb_samples_araujo2021, 
                                  name_func="araujo2021", 
                                  return_time=True)
    sigmas_1_araujo2021.append(sigma_1_araujo2021.item())
    times.append(time)
print("Diff Sedghi2019 with Araujo2021 conv ", np.mean(sigmas_1_araujo2021[burn:]) - sigma_1_reference_sedghi2019,
      "\nDiff Ryu2019 with Araujo2021 conv", np.mean(sigmas_1_araujo2021[burn:]) - sigma_1_reference_ryu2019, 
      "\nMean time", np.mean(times[burn:]))

Diff Sedghi2019 with Araujo2021 conv  271.8495864868164 
Diff Ryu2019 with Araujo2021 conv 274.9720458984375 
Mean time 0.0016489505767822265


# Test Singla2021 on convolutional layer

In [7]:
n_iter_singla2021 = 50
nb_reps = 10
burn = 10
sigmas_1_singla2021, times = [], []
for _ in range(nb_reps + burn):
    sigma_1_singla2021, time = estimate(kernel, 
                                  n=input_size_n, 
                                  n_iter=n_iter_singla2021, 
                                  name_func="singla2021", 
                                  return_time=True)
    sigmas_1_singla2021.append(sigma_1_singla2021.item())
    times.append(time)
print("Diff Sedghi2019 with Singla2021conv", np.mean(sigmas_1_singla2021[burn:]) - sigma_1_reference_sedghi2019,
      "\nDiff Ryu2019 with Singla2021 conv", np.mean(sigmas_1_singla2021[burn:]) - sigma_1_reference_ryu2019, 
      "\nMean time", np.mean(times[burn:]))

Diff Sedghi2019 with Singla2021conv 97.87226562500001 
Diff Ryu2019 with Singla2021 conv 100.9947250366211 
Mean time 0.024326324462890625


# Test Gram iteration on convolution layer

In [8]:
n_iter_gram = 5
nb_reps = 10
burn = 10
sigmas_1_gram, times = [], []
for _ in range(nb_reps + burn):
    sigma_1_gram, time = estimate(kernel, 
                                  n=input_size_n, 
                                  n_iter=n_iter_gram, 
                                  name_func="ours", 
                                  return_time=True)
    sigmas_1_gram.append(sigma_1_gram.item())
    times.append(time)
print("Diff Sedghi2019 with Gram iteration conv", np.mean(sigmas_1_gram[burn:]) - sigma_1_reference_sedghi2019,
      "\nDiff Ryu2019 with Gram iteration conv", np.mean(sigmas_1_gram[burn:]) - sigma_1_reference_ryu2019, 
      "\nMean time", np.mean(times[burn:]))

Diff Sedghi2019 with Gram iteration conv 0.0429534912109375 
Diff Ryu2019 with Gram iteration conv 3.1654129028320312 
Mean time 0.0011824369430541992


# Compute spectral norm of convolutional layers of ResNet18

This code is related to Section 5.3 in paper.

In [ ]:
from torchvision import models

n_iter_name = {
               "ours":7,
               "singla2021":50,
               "ryu2019" : 100,
               "araujo2021" : 20,
               "sedghi2019":None}
func_names = [
              "ryu2019",
              #"sedghi2019", # commented because it takes a while
              "araujo2021",
              "singla2021",
              "ours",
]
model_resnet_18 = models.resnet18(pretrained=True).cuda().eval()



inp_shape = (224, 224)
lip_tot = {name : 1 for name in func_names}
times_tot = {name : 0 for name in func_names}

with torch.no_grad():
    for name, module in model_resnet_18.named_modules():
        print()
        is_downsample = name.endswith("downsample")
        is_regular_conv = "conv" in name
        is_max_pool = "MaxPool2d" in module.__class__.__name__
        if is_max_pool:
            stride = module.stride
            inp_shape = (inp_shape[0] // stride, inp_shape[1] // stride)
        if is_downsample:
            # dowsampling layer in residual connection
            conv, bn = module[0], module[1]
            lip_bn = (bn.weight.detach() / bn.running_var).max().item()
            inp_shape = (inp_shape[0] * stride, inp_shape[1] * stride)
        elif is_regular_conv:
            conv = module
            stride = conv.stride[0]
            lip_bn = 1.0
        if is_downsample or is_regular_conv:
            param = conv.weight.clone().detach()
            out_channels, in_channels, H, W = param.shape
            for name in func_names:
                bound, curr_time = estimate(param,
                                            inp_shape[0],
                                            n_iter_name[name],
                                            name,
                                            return_time=True)
                bound = bound.item()
                print(name, "conv weight dim", param.shape, "n", inp_shape[0], "bound", bound, "time", curr_time)
                if is_downsample:
                    lip_tot[name] += bound* lip_bn
                else:
                    # Lipschiz of batch norm cancels in ratio
                    lip_tot[name] *= bound
                times_tot[name] += curr_time
                print("Total Lipschitz bound", lip_tot[name], "total time", times_tot[name], "\n")
            inp_shape = (inp_shape[0] // stride, inp_shape[1] // stride)


# Total Lipschitz ratio bound 

In [10]:
lip_tot_ref = lip_tot["ryu2019"]
for name in func_names:
    lip_tot[name] /= lip_tot_ref
print("Total Lipschitz ratio", lip_tot)
print("Total times", times_tot)

Total Lipschitz ratio {'ryu2019': 1.0, 'araujo2021': 30387172993.15906, 'singla2021': 88.44895431546131, 'ours': 1.4790599120830212}
Total times {'ryu2019': 0.8379137516021729, 'araujo2021': 0.02653217315673828, 'singla2021': 0.5859165191650391, 'ours': 0.04935169219970703}
